In [1]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes,processedNotes
from src.utils.utils import convertNotesToChord, convertChordTofingerPositions
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool
import copy
import time

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [2]:
midiFilePath = "asset/Aguado_12valses_Op1_No2.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

GuitarNotes = midiToGuitarNotes(midiFilePath)

GuitarNotes

[{'notes': [48, 52, 55], 'beat': 0.375},
 {'notes': [57], 'beat': 0.5},
 {'notes': [55], 'beat': 1.0},
 {'notes': [64], 'beat': 1.5},
 {'notes': [48, 60], 'beat': 2.0},
 {'notes': [55], 'beat': 2.5},
 {'notes': [52, 55], 'beat': 3.0},
 {'notes': [47, 53, 55], 'beat': 3.375},
 {'notes': [57], 'beat': 3.5},
 {'notes': [55], 'beat': 4.0},
 {'notes': [65], 'beat': 4.5},
 {'notes': [47, 62], 'beat': 5.0},
 {'notes': [55], 'beat': 5.5},
 {'notes': [53, 55], 'beat': 6.0},
 {'notes': [48, 55, 64], 'beat': 6.375},
 {'notes': [65], 'beat': 6.5},
 {'notes': [64], 'beat': 7.0},
 {'notes': [47, 56, 62], 'beat': 7.5},
 {'notes': [45, 57, 60], 'beat': 7.875},
 {'notes': [62], 'beat': 8.0},
 {'notes': [60], 'beat': 8.5},
 {'notes': [40, 55, 58], 'beat': 9.0},
 {'notes': [41, 53, 57], 'beat': 9.5},
 {'notes': [62], 'beat': 10.0},
 {'notes': [42, 51, 60], 'beat': 10.5},
 {'notes': [43, 50, 60], 'beat': 10.75},
 {'notes': [57], 'beat': 11.25},
 {'notes': [55], 'beat': 11.5},
 {'notes': [54], 'beat': 11.7

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [3]:
# 设定各弦音高
guitar_string_list = createGuitarStrings(["e", "b", "G", "D", "A", "E1"])
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(20)
handPoseRecordPool.append(handPoseRecord)

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [4]:
for i in range(len(GuitarNotes)):
    start_time = time.time()
    guitarNote = GuitarNotes[i]
    notes = guitarNote["notes"]
    beat = guitarNote["beat"]
    notes = processedNotes(notes)
    
    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)
    
    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
        
    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        fingerPositionsList = convertChordTofingerPositions(chord)
        for fingerPositions in fingerPositionsList:
            # Iterate through all recorders in the record pool. 遍历记录池中的所有记录器。
            for handPoseRecord in handPoseRecordPool.preHandPoseRecordPool:
                oldhand = handPoseRecord.currentHandPose()
                # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
                hand, entropy = oldhand.generateNextHands(
                    guitar, fingerPositions)
                if hand is None:
                    continue
                newHandPoseRecord = copy.deepcopy(handPoseRecord)
                newHandPoseRecord.addHandPose(hand, entropy, beat)
                handPoseRecordPool.append(newHandPoseRecord)
    
    end_time = time.time()
    print(f"当前beat是{beat}，当前有{len(handPoseRecordPool.curHandPoseRecordPool)}个rerocder，耗时{end_time-start_time:.2f}秒。")


# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(f"output/{filename}.json")

当前beat是0.375，当前有3个rerocder，耗时0.00秒。
当前beat是0.5，当前有4个rerocder，耗时0.00秒。
当前beat是1.0，当前有4个rerocder，耗时0.01秒。
当前beat是1.5，当前有3个rerocder，耗时0.01秒。
当前beat是2.0，当前有11个rerocder，耗时0.01秒。
当前beat是2.5，当前有5个rerocder，耗时0.02秒。
当前beat是3.0，当前有8个rerocder，耗时0.02秒。
当前beat是3.375，当前有17个rerocder，耗时0.03秒。
当前beat是3.5，当前有4个rerocder，耗时0.05秒。
当前beat是4.0，当前有2个rerocder，耗时0.01秒。
当前beat是4.5，当前有2个rerocder，耗时0.01秒。
当前beat是5.0，当前有8个rerocder，耗时0.01秒。
当前beat是5.5，当前有4个rerocder，耗时0.03秒。
当前beat是6.0，当前有11个rerocder，耗时0.03秒。
当前beat是6.375，当前有20个rerocder，耗时0.22秒。
当前beat是6.5，当前有3个rerocder，耗时0.13秒。
当前beat是7.0，当前有3个rerocder，耗时0.02秒。
当前beat是7.5，当前有5个rerocder，耗时0.02秒。
当前beat是7.875，当前有6个rerocder，耗时0.04秒。
当前beat是8.0，当前有17个rerocder，耗时0.05秒。
当前beat是8.5，当前有3个rerocder，耗时0.16秒。
当前beat是9.0，当前有20个rerocder，耗时0.03秒。
当前beat是9.5，当前有2个rerocder，耗时0.03秒。
当前beat是10.0，当前有6个rerocder，耗时0.02秒。
当前beat是10.5，当前有5个rerocder，耗时0.02秒。
当前beat是10.75，当前有10个rerocder，耗时0.08秒。
当前beat是11.25，当前有5个rerocder，耗时0.09秒。
当前beat是11.5，当前有1个rerocder，耗时0.04秒。
当前beat是11.75，当前有4个rerocder

## 初始化blender中的参数 Initialize Environment Value in Blender

Measure the coordinates of several key points on the guitar in Blender to facilitate subsequent calculations. 在blender中测量吉他的几个关键点的坐标，以便后续计算。

The three key points are: 三个关键点是:
- The coordinate of the first fret of the first string. 吉它的第1弦空品的坐标
- The coordinate of the twelfth fret of the first string while pressing the string. 第1弦第12品按弦状态下的坐标
- The coordinate of the first fret of the sixth string. 第6弦空品的坐标。

Once these three points are determined, it is equivalent to defining a plane. This plane is the one that the left hand must fall into when pressing the strings. 因为当这三个点确定以后，就相当于确定了一个平面，这个平面就是左手按弦时必定会落于其中的平面。

Then the distance between the string and the fretboard when the string is open, which is used to calculate the coordinates of the fingers when not pressing the strings. This distance should be slightly higher than the measured value, as one needs to leave some space to prevent the fingers from touching the strings. 然后就是一个空弦时弦与指板的距离，这个是用来计算非按弦状态时手指坐标的。这个距离要比测量值高一点，毕竟人还要留下一点空音防止手指触弦。

Finally, there is the BPM (beats per minute), which is used to calculate the duration of notes. 最后面就是BPM(beat per minute)了，这个是用来计算音符的时长的。

In [7]:
from numpy import linalg, array
from src.utils.fretDistanceDict import FRET_DISTANCE_DICT
from src.utils.tab2Space import find_transform_matrix, normalVector

# define three key points on the first plane p. 定义第一个平面p上的三个关键点。
# The p plane can be understood as the tab surface.我们可以把p平面理解为tab谱面。
# p0 is the open string of the first string, p1 be the 12th fret of the first string, and p2 be the open string of the sixth string. 第一弦空品就是p0，第一弦12品就是p1，第六弦空品就是p2。
# the value of x at the 12th fret is defined as 0.5, because the 12th fret is just half the length of the string. 第12品的x值定为0.5，因为第12品处刚好是整弦长的一半。
# the value of y of the sixth string is defined as 5, because its index is 5. 第六弦的y值定义为5，因为从0开始排下来它序列号为5。
p0 = array([0, 0, 0])
p1 = array([0.5, 0, 0])
p2 = array([0, 5, 0])

# The coordinates of the three key points on the guitar fretboard. blender文件中，吉它指板上的三个关键点的坐标
q0 = array([0.60115, -0.20122, 1.7271])
q1 = array([0.17146, -0.3363, 1.397])
q2 = array([0.30514, -0.28917, 1.4956])

fret_board_normal_vector = normalVector(q0, q1, q2)

fret_open_move = [0.000587, 0.002679, 0.001117]

# the distance between the string and the fretboard. 当手指松开时，相当于在指板z方向上移动了fret_open_move的距离
fret_open_move = array([0.000587, 0.002679, 0.001117])
fret_open_z = linalg.norm(fret_open_move)
fret_open_distance = fret_open_z * fret_board_normal_vector

# rotation matrix and translation vector from p plane to q plane. p平面到q平面的转换矩阵和平移向量。
rotation_matrix, translation_vector = find_transform_matrix(p0, p1, p2, q0, q1, q2)

# beats per minute. 乐曲的演奏速度
BPM = 100
# frames per second. blender文件中的动画帧率
FPS = 24

recorder = f"output/{filename}.json"
animation = f"output/{filename}_animation.json"

In [9]:
from src.animate.animate import hand2Animation
hand2Animation(recorder, animation, BPM, FPS, rotation_matrix, translation_vector, fret_open_distance, FRET_DISTANCE_DICT)